# Creating command-line Tools

In [ ]:
from ctapipe.core import Tool, Component
from ctapipe.core.traits import (Integer, Float, List, Dict,Unicode, TraitError, observe)
import logging
from time import sleep

see https://github.com/ipython/traitlets/blob/master/examples/myapp.py

## Setup:

Create a few `Component`s that we will use later in a `Tool`:

In [ ]:
class MyComponent(Component):
    """ A Component that does stuff """
    value = Integer(default_value=-1, help="Value to use").tag(config=True)

    def do_thing(self):
        self.log.debug("Did thing")

# in order to have 2 of the same components at once
class SecondaryMyComponent(MyComponent):
    pass


class AdvancedComponent(Component):
    """ An advanced technique """

    value1 = Integer(default_value=-1, help="Value to use").tag(config=True)
    infile = Unicode(help="input file name").tag(config=True)
    outfile = Unicode(help="output file name").tag(config=True)

    @observe('outfile')
    def on_outfile_changed(self, change):
        self.log.warning("Outfile was changed to '{}'".format(change))

In [ ]:
MyComponent()

In [ ]:
AdvancedComponent()

## Now create an executable Tool that contains the Components

In [ ]:
class MyTool(Tool):
    name="mytool"
    description="do some things and stuff"
    aliases = Dict(dict(infile='AdvancedComponent.infile',
                        iterations='MyTool.iterations'))

    # Which classes are registered for configuration
    classes = List([MyComponent, AdvancedComponent, SecondaryMyComponent])

    # local configuration parameters
    iterations = Integer(5,help="Number of times to run",allow_none=False).tag(config=True)

    def setup_comp(self):
        # when constructing Components, you must add them to the 
        # list of registered instances using add_component. This allows
        # the full configuration to be tracked
        self.comp = self.add_component(MyComponent(parent=self))
        self.comp2 = self.add_component(SecondaryMyComponent(parent=self))
        

    def setup_advanced(self):
        self.advanced = self.add_component(AdvancedComponent(parent=self))

    def setup(self):
        self.setup_comp()
        self.setup_advanced()

    def start(self):
        self.log.info("Performing {} iterations...".format(self.iterations))
        for ii in range(self.iterations):
            self.log.info("ITERATION {}".format(ii))
            self.comp.do_thing()
            self.comp2.do_thing()
            sleep(0.1)
            
    def finish(self):
        self.log.warning("Shutting down.")
    

## Get Help info

The following allows you to print the help info within a Jupyter notebook, but this same inforamtion would be displayed if the user types:
```
  mytool --help
```

In [ ]:
tool=MyTool()

In [ ]:
tool.print_help()

The following  is equivalant to the user typing `mytool --help-all`

In [ ]:
tool.print_help(classes=True)

## Run the tool

here we pass in argv since it is a Notebook, but if argv is not specified it's read from `sys.argv`, so the following is the same as running:

```sh
mytool --log_level=INFO --infile bork.txt --iterations=3
```

In [ ]:
tool.run(argv=[])

In [ ]:
tool.log_format = "%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s" 
tool.run(argv=['--log-level','INFO','--infile','bork.txt','--iterations','3'])

here we change the log-level to DEBUG:

In [ ]:
tool.run(argv=['--log-level','DEBUG','--infile','bork.txt'])

you can also set parameters directly in the class, rather than using the argument/configfile parser. This is useful if you are calling the Tool from a script rather than the command-line

In [ ]:
tool.iterations = 1
tool.log_level = 0
tool.run('')

see what happens when a value is set that is not of the correct type:

In [ ]:
try:
    tool.iterations = "badval"
except TraitError as E:
    print("bad value:",E)

Example of what happens when you change a parameter that is being "observed" in a class. It's handler is called:

In [ ]:
tool.advanced.outfile = "Another.txt"

we see that the handler for `outfile` was called, and it receive a change dict that shows the old and new values.

create a tool using a config file:

In [ ]:
!cat Tools.json

In [ ]:
tool2 = MyTool()

In [ ]:
tool2.run(argv=['--config','Tools.json'])

In [ ]:
print(tool2.advanced.infile)

In [ ]:
print(tool2.config)

In [ ]:
tool2.is_setup

In [ ]:
tool3 = MyTool()

In [ ]:
tool3.is_setup

In [ ]:
tool3.initialize(argv=[])

In [ ]:
tool3.is_setup

In [ ]:
tool3

In [ ]:
tool

In [ ]:
tool.comp2

## Getting the configuration of an instance

In [ ]:
tool.get_current_config()

In [ ]:
tool.iterations = 12
tool.get_current_config()

## Writing a Sample Config File

In [ ]:
print(tool.generate_config_file())